<a href="https://colab.research.google.com/github/claudelepere/ML_GitHub/blob/main/BERT_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Inference

Let's test the model on new datapoints:

In [1]:
# Do not use conda in Colab notebook, do not try to install with conda, use pip

!pip install -q huggingface_hub
!pip install -q datasets

import json
import os
import pandas as pd
import torch

from datasets        import DatasetDict
from google.colab    import files, userdata
from huggingface_hub import login, hf_hub_download
from transformers    import AutoModelForSequenceClassification, AutoTokenizer


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
### Check the device: no need to use T4 GPU for inference

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"device: {device}")


device: cpu


In [3]:
### Hugging Face authenticate

os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
hf_token               = os.environ.get('HF_TOKEN')
login(token = hf_token)

# Verify
!huggingface-cli whoami


Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


claudelepere


In [4]:
### Model repo name on Hugging Face Hub

HF_Hub_name = "claudelepere/skill_classification"


In [5]:
### Download the labels from the dataset repo

file_path = hf_hub_download(
    repo_id                = HF_Hub_name,
    filename               = "labels.json",
    repo_type              = "dataset",
    local_dir              = "./"
    )

with open(file_path, 'r') as f:
    labels = json.load(f)

print(f"labels: {type(labels)} len={len(labels)}\n{labels}")


labels.json:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

labels: <class 'list'> len=206
['178', '180', '181', '182', '183', '188', '189', '190', '196', '198', '199', '203', '204', '205', '206', '208', '209', '210', '214', '215', '216', '263', '265', '266', '267', '268', '270', '271', '272', '273', '274', '275', '276', '277', '278', '283', '284', '285', '286', '288', '289', '291', '292', '293', '297', '298', '300', '301', '302', '303', '304', '308', '309', '310', '311', '312', '313', '314', '315', '316', '317', '318', '320', '321', '322', '324', '358', '359', '365', '366', '367', '369', '372', '378', '379', '380', '381', '382', '384', '385', '396', '397', '398', '399', '400', '401', '402', '403', '404', '405', '406', '407', '410', '411', '610', '611', '612', '613', '614', '615', '616', '618', '630', '631', '633', '636', '637', '660', '661', '662', '668', '670', '672', '673', '676', '677', '678', '681', '682', '683', '693', '694', '696', '697', '698', '699', '700', '701', '702', '703', '704', '705', '706', '708', '717', '718', '719', '720', '7

In [6]:
### Get the filtered skills, those present in labels, in order to have their Value

files.upload()

skill_df          = pd.read_csv('skills.csv')
skill_df['Id']    = skill_df['Id'].astype(str)
skill_df['Value'] = skill_df['Value'].astype(str)
labels            = [str(label) for label in labels]
filtered_skill_df = skill_df[skill_df['Id'].isin(labels)]

print(f"filtered_skill_df: {type(filtered_skill_df)} shape={filtered_skill_df.shape}\n{filtered_skill_df}")


Saving skills.csv to skills.csv
filtered_skill_df: <class 'pandas.core.frame.DataFrame'> shape=(206, 3)
      Id  SkillTypeId                            Value
61   178            8                             Abap
62   180            8                    {.NET}ASP.NET
63   181            8                         {.NET}C#
64   182            8                          C / C++
65   183            8                            Cobol
..   ...          ...                              ...
294  805           10     {Management Frameworks}PMBOK
295  806           10  {Management Frameworks}Prince 2
296  807           10  {Management Frameworks}UP / RUP
297  808           10                            Togaf
298  809           10                        Waterfall

[206 rows x 3 columns]


In [7]:
### Download the model and the tokenizer from the model repo

model = AutoModelForSequenceClassification.from_pretrained(
    HF_Hub_name,
    num_labels = len(labels)
)

tokenizer = AutoTokenizer.from_pretrained(HF_Hub_name)


config.json:   0%|          | 0.00/7.54k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/439M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

In [8]:
### New texts to classify

text_None = "I'm happy, I can finally train a model for multi-label classification."

# NL
text_323697 = "Talencia Consulting - Senior Full Stack Developer (Java & Angular) Full Stack, DevOps, AKS, OAuth, API Talencia Consulting Voor een klant van Talencia ben ik opzoek naar een Senior Full Stack Developer (Java & Angular) Job beschrijving Als Developer zal je een bestaand team toevoegen en meewerken aan de buitbouw van webapplicaties op Azure. Dit is om bestaande applicaties te vervangen die end-of-live zijn. Het project is al in volle realisatie. Profiel Zeer goede kennis van Java en Angular Goede kennis van Azure DevOps, AKS,.. is een grote pluspunt Kennis van Docker/ SQL/ OAuth/PWA/ RESTful API is vereist Taal: Nederlands met kennis van Engels Extra informatie Teamspeler met ervaring in Agile methodiek is vereist. Als je meer informatie wilt en dit klinkt interessant voor u, aarzel dan niet om uw meest recente CV door te sturen. Het kan zijn dat ik niet beschik over uw meest recente CV en dat ik daarom u deze opportuniteit doorstuur dat niet geschikt is voor u. Als u iemand kent dat deze missie interessant zou vinden mag u deze vacature doorsturen. Met vriendelijke groeten,"
# 142,189,190,754,208,794,676,811,812,139,138
# SkillTypeId 7       : true: 142 preds: 142, 170
# SkillTypeId 8, 9, 10: true: 189, 190, 754, 208, 794, 676, 811, 812 preds:
# SkillTypeId 11      : true: 139, 138

# EN
text_23553 = "Cream Consulting - Business Analyst Insurance, Property&Casualty (P&C) Cream Consulting Today we are looking for Business Analyst to extend our team specialized in Business Process Improvement. You will join a multi-lingual team focusing on P&C business. In that position, you will define and clarify the role and responsibilities. You will be responsible for leading business process reviews where you will identify and implement innovative solutions. You will define and document the new processes and systems to meet our client's business objectives. What the job is all about? Implement new solutions for P&C insurance companies Write and validate the “as is” Plan the analysis of organization's strategic business needs Understand End Users needs Develop process modeling and design Follow up projects and monitor development Organize and conduct testing What Cream is looking for? At least 5 years of experience in Business Analysis in insurance business Analysis know-how Fluent editorial Good knowledge of Property & Casualty insurance business to interface with all stakeholders Functional knowledge of insurance products Good communication skills Fluent English, good knowledge in French and/or Dutch is a plus"
#356,168,149,795,802,137,139,138,353
# SkillTypeId 7 : true: 356, 168, 149 preds: 142, 170
# SkillTypeId 8 :
# SkillTypeId 9 :
# SkillTypeId 10: true: 795, 802
# SkillTypeId 11: true: 137, 139, 138
# SkillTypeId 12: true: 353

text_323611 = "Atcon Global - Project Management Officer / PMO team management Atcon Global For one of our clients, we are looking for an experienced Project Management Officer (PMO) / Project Manager (PM) for permanent employment in the Flanders region. Your role? As a PMO, you will play a crucial role in setting up and improving our project management processes. You will not only be responsible for developing PM standards, but also for carrying out projects independently as a Project Manager. Your duties and responsibilities will include: Developing PMO and project management standards Executing and managing complex digital projects Oversee project progress and report to senior management Follow-up of project budgets, project selection, capacity planning and resource management Coaching and training project managers Identifying and managing project risks Promote continuous improvement in the project management domain Collaborate with stakeholders and external partners Who are we looking for? Bachelor's or master's degree 5+ years in a similar role in a dynamic organization Expertise in project management methods (Agile, Scrum, Lean, Kanban) Strong analytical and problem-solving skills Excellent communication and stakeholder management Experience in team management with clear objectives Proactive, Hands-on mentality and result-oriented Fluent in Dutch and English; French is a plus What's on offer? A dynamic and varied role in a growing, ambitious and innovative company Numerous opportunities for personal growth and career development A competitive salary with customizable benefits A friendly, collegial working atmosphere Flexible working hours, possibility to work from home"
# 171,170,794,800,798,797,138,139,352
# SkillTypeId 7       : true: 170, 171           preds: 142, 170
# SkillTypeId 8, 9, 10: true: 794, 800, 798, 797 preds:
# SkillTypeId 11      : 138, 139
# SkillTypeId 12      : 352

text_323526 = "Vivid Resourcing - Chief Technology Officer CTO, reliability, business goals Vivid Resourcing We're partnered with a leading sustainability-oriented company near Brussels, aiming to combat high pollution rates worldwide. They are currently working on a unique application that rewards workers for reducing their carbon footprint, whilst also maintaining and even improving profits. Together we are seeking a visionary Chief Technology Officer (CTO) who aligns with their mission and ambitions. The ideal candidate will possess a strong hands-on technical background, proven management experience, and strong business acumen. This role requires a strategic thinker who can drive technological direction and support the company's growth objectives of transitioning from a scale-up to an established business entity, so any past experience leading teams in this manner would go a long way. Key responsibilities Develop and execute the company's technological vision and strategy Lead and mentor a team of engineers and technologists Oversee all technical aspects of the company, ensuring alignment with business goals Drive innovation in regenerative sustainable technologies and carbon measurement systems Collaborate with cross-functional teams to integrate technology solutions Ensure the reliability, security, and scalability of technological infrastructures Foster a culture of continuous improvement and technical excellence Qualifications Experience leading a team within a small to medium sized company Strong technical background in software development/data analytics/system architecture Bachelor's or Master's degree in either an IT or Business related field Experience in the agriculture or environmental sectors is a plus Proven management skills with the ability to lead, communicate and inspire a diverse team Excellent business acumen and strategic thinking Strong problem-solving skills and the ability to make informed decisions in a fast-paced environment Offer Taking charge of a genuinely impactful product, using your direction for the good of the environment Complete responsibility over a technical team, with management responsibilities Up to 110,000 EUR gross for experienced applicants, which can then be increased further Full benefits package including mobility costs Flexible hybrid work Inclusive work environment If this role interests you, attach a CV and apply today!"
# 175,169,139,352
# SkillTypeId 7       : true: 169, 175 preds: 142, 170
# SkillTypeId 8, 9, 10: true:
# SkillTypeId 11      : true: 139
# SkillTypeId 12      : true: 352

# FR
text_323517 = "AG Insurance - Technical Architect REST API, DevOps AG Insurance Un lieu de travail où vous pouvez prendre des initiatives. Où tout le monde vous encourage à montrer ce dont vous êtes capable. Où on vous encourage à vous développer. Et où vous construisez l'avenir avec vos collègues. Vous avez de l'audace et le client est au centre de vos préoccupations ? Alors, à très bientôt chez AG ! Notre compagnie compte environ 700 spécialistes IT. Autant de collègues géniaux qui contribuent chaque jour à la (r)évolution technologique chez AG. Le département Information System (IS) est responsable du développement des applications supportant le business assurance, ainsi que des applications transversales dans les domaines suivants : In & Outbounds, Referentials & Financials, sans oublier des centres d'expertise technologique pour le développement des Front-Ends, pour la mise en place d'APIs et s... Et si c'était votre prochain job ? Nous recherchons un(e) .NET Solution Architect pour renforcer ce département. Vous proposez des solutions techniques adaptées aux besoins et aux contraintes des projets et conformes à la vision IT d'AG. En innovant et en améliorant ce qui existe déjà, vous créez également une valeur ajoutée pour AG. Pendant la phase architecturale, vous choisissez les solutions en concertation avec les autres architectes techniques et fonctionnels, et avec toutes les autres parties concernées (autres équipes, partenaires externes, collègues d'Infrastructure & Operations, etc.). Vous concevez l'architecture technique des applications sur la base des exigences fonctionnelles et non fonctionnelles. Nous comptons également sur vous pour le support technique, du développement à la production de l'application. Vous assurez le suivi de l'implémentation de vos solutions, en coopération avec les équipes de développement. Vous vous assurez ensuite que la solution mise en œuvre est conforme à l'architecture proposée. Si nécessaire, vous apportez des corrections. Vous accompagnez et coachez vos collègues pour les aider à monter en compétence dans le domaine. Ce poste vous intéresse ? Qu'attendez-vous pour postuler ? Nos recruteurs examineront avec vous quelle division de notre département IT et quelle équipe vous conviendront le mieux. Vous vous reconnaissez dans ce profil ? Les nouvelles technologies ? Vous en suivez de près l'évolution et les tendances en architecture IT. Vous avez une expérience solide dans les domaines suivants : Technical Architecture & Design Patterns REST API, .NET Framework & .NET 6 and up Source control tooling like Azure DevOps & GitHub Microsoft Azure Cloud SQL Server Vous êtes autonome et prenez les choses en main ? Bien sûr ! Mais vous savez remonter les informations ou demander du support si nécessaire. Team-player dans l'âme, vous partagez naturellement votre savoir-faire avec vos collègues. La qualité du travail, dans ses moindres détails, c'est votre cheval de bataille. Vous le documentez avec précision, tant pour le support technique de votre équipe que pour les utilisateurs d'autres teams. Communiquer en français et/ou en néerlandais, à l'oral et à l'écrit ? Un jeu d'enfant pour vous ! Votre anglais est à la hauteur ? Encore mieux ! Un diplôme supérieur en informatique et/ou plus de 2 ans d'expérience dans une fonction similaire ? Foncez ! Vous n'allez pas rater ça ? Un super job chez le leader sur le marché de l'assurance. Et comme nous cherchons à encore mieux servir nos clients, nous comptons sur vous pour nous y aider. Un environnement de travail moderne dans tous les sens du terme : physique, digital et organisationnel. En d'autres termes, des heures de travail flexibles, la possibilité de télétravailler jusqu'à 3 jours par semaine et le matériel IT adéquat pour travailler de la maison. Une équipe enthousiaste et dynamique, notée 10/10 pour l'ambiance et la convivialité. La possibilité de vous perfectionner en continu, grâce à un large éventail de formations. Idéal pour apprendre toutes les compétences qui vous aideront à faire évoluer votre carrière. Une véritable carrière. En effet, travailler chez AG, c'est bien plus qu'un job. Envie d'explorer de nouveaux horizons après quelque temps ? Nous vous guidons et vous encourageons à exploiter tous vos talents à fond. Des bonnes vibrations, grâce à un vaste programme de bien-être bourré d'activités sportives et d'ateliers inspirants. De quoi vous sentir (encore) plus épanoui(e) dans votre travail et votre vie. Un lieu où tout le monde se sent bienvenu et a des chances égales de s'épanouir. Où votre avenir n'est pas déterminé par votre origine, votre âge, votre genre, votre orientation sexuelle ou votre handicap, mais par votre talent et vos compétences. Et comme l'argent, ça compte aussi : un package salarial attrayant. Vous pouvez même composer vous-même une partie de votre package, car personne ne sait mieux que vous ce dont vous avez besoin. Lancez-vous ! Emballé(e) ? > Postulez aujourd'hui encore ! Nous nous ferons un plaisir de vous aider si vous avez besoin de soutien avant ou pendant le processus de sélection."
# 146,636,676,668,670,300,812,138,137
# SkillTypeId 7       : true: 146 preds: 142, 170, 689
# SkillTypeId 8, 9, 10: true: 636, 676, 668, 670, 300, 812
# SkillTypeId 11      : true: 138, 137


text = text_23553


In [9]:
### Classify

inputs = tokenizer(
    text,
    return_tensors='pt',
    truncation=True,
    padding='max_length',
    max_length=512
)


The logits that come out of the model are of shape (batch_size, num_labels). As we are only forwarding a single sentence through the model, the `batch_size` equals 1. The logits is a tensor that contains the (unnormalized) scores for every individual label.

In [10]:
### Perform the forward pass: logits are raw scores for each label

with torch.no_grad():
  outputs = model(**inputs)
  logits  = outputs.logits

#print(f"logits: {type(logits)} shape={logits.shape}\n{logits}")


To turn them into actual predicted labels, we first apply a sigmoid function independently to every score, such that every score is turned into a number between 0 and 1, that can be interpreted as a "probability" for how certain the model is that a given class belongs to the input text.

Next, we use a threshold (typically, 0.5) to turn every probability into either a 1 (which means, we predict the label for the given example) or a 0 (which means, we don't predict the label for the given example).

In [11]:
### Squash logits to [0, 1] probabilities:

probs = torch.sigmoid(logits)
#print(f"probs: {type(probs)} shape={probs.shape}\n{probs}")

probs = probs.squeeze()
#print(f"probs: {type(probs)} shape={probs.shape}\n{probs}")


In [12]:
### Convert probabilities into binary predictions

threshold      = 0.15
preds          = (probs >= threshold).int()
#print(f"preds: type={type(preds)} shape={preds.shape}\n{preds}")

sorted_indices = torch.argsort(probs, descending=True)
#print(f"sorted_indices: {type(sorted_indices)} shape={sorted_indices.shape}\n{sorted_indices}")
sorted_probs   = probs[sorted_indices]
#print(f"sorted_probs: {type(sorted_probs)} shape={sorted_probs.shape}\n{sorted_probs}")
sorted_labels  = [labels[i] for i in sorted_indices]
#print(f"sorted_labels: {type(sorted_labels)} shape={sorted_labels.shape}\n{sorted_labels}")
sorted_preds   = preds[sorted_indices]
#print(f"sorted_preds: {type(sorted_preds)} shape={sorted_preds.shape}\n{sorted_preds}")

print(f"skill\tprob (thr={threshold})\tpred\tValue")
for label, prob, pred, Value in zip(sorted_labels, sorted_probs, sorted_preds, filtered_skill_df['Value']):
  print(f"{label}\t{prob.item():.4f}\t\t{int(pred.item())}\t{Value}")

print()

print(f"skill\tprob (thr={threshold})\tpred\tValue")
if (preds == 0).all():
  print("All predictions are 0, no skills found")
else:
  for label, prob, pred, Value in zip(sorted_labels, sorted_probs, sorted_preds, filtered_skill_df['Value']):
    if pred == 1:
      print(f"{label}\t{prob.item():.4f}\t\t{int(pred.item())}\t{Value}")


skill	prob (thr=0.15)	pred	Value
676	0.2445		1	Abap
794	0.2413		1	{.NET}ASP.NET
636	0.1794		1	{.NET}C#
754	0.1710		1	C / C++
274	0.1499		0	Cobol
385	0.1460		0	HTML / HTML5
289	0.1441		0	Java / J2EE
189	0.1409		0	JavaScript
313	0.1353		0	Oracle Forms / Reports
284	0.1353		0	Perl
800	0.1312		0	PHP
812	0.1282		0	PL1
798	0.1269		0	{SQL}Oracle PL/SQL
291	0.1268		0	RPG
382	0.1261		0	SQL
324	0.1227		0	{SQL}Microsoft SQL Server
208	0.1226		0	{SQL}MySQL / MariaDB
190	0.1194		0	{.NET}VB.NET
322	0.1192		0	Visual Basic / VBA
795	0.1186		0	WinDev / WebDev
396	0.1159		0	Ruby
303	0.1134		0	Shell
183	0.1127		0	Python
787	0.1126		0	{Java / J2EE}JBoss (WildFly) / Tomcat
722	0.1126		0	Mobile devices
209	0.1125		0	{Mobile devices}Android
292	0.1121		0	{Mobile devices}iOS
769	0.1121		0	.NET
198	0.1110		0	{Java / J2EE}Grails / Groovy
397	0.1108		0	{Java / J2EE}GWT
814	0.1093		0	{Java / J2EE}JPA / Hibernate
830	0.1091		0	{Java / J2EE}JSF
203	0.1089		0	{Java / J2EE}JSP / Servlets
827	0.1078		0	{Java / J2EE}JU